In [2]:
%load_ext autoreload
%autoreload 2
%pylab inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Populating the interactive namespace from numpy and matplotlib


In [3]:
%load_ext Cython
%load_ext line_profiler
%load_ext memory_profiler

In [4]:
import sys, argparse, logging
import pymssql
import _mssql
from datetime import datetime, time
from decimal import Decimal
import pymongo
from pymongo import MongoClient
from tqdm import tqdm, tnrange, tqdm_notebook
import pandas as pd
from tmqr.settings import *
from tmqrfeed.assetsession import AssetSession
import pickle
import pytz
try:
    from tmqr.settings_local import *
except:
    pass
import pyximport

pyximport.install(setup_args={"include_dirs": np.get_include()})
from tmqrfeed.quotes.compress_daily_ohlcv import compress_daily
import os

from tmqrfeed.assetsession import AssetSession
from tmqrfeed.quotes.dataframegetter import DataFrameGetter
from tmqrfeed.datafeed import DataFeed

In [5]:
client = MongoClient(MONGO_CONNSTR)

In [7]:
db = client[MONGO_DB]


In [121]:
data = db[SRC_INTRADAY].find_one({'tckr': 'US.F.CL.Q12.120720', 'dt': pd.Timestamp('2012-07-16 00:00:00.000Z')})
df = pickle.loads(data['ohlc'])

In [123]:
tz = pytz.timezone("US/Pacific")
df = df.tz_convert(tz)

In [124]:
tz.zone

'US/Pacific'

In [125]:
d = tz.localize(pd.Timestamp('2012-07-16 16:59:00.000'))


In [126]:
len(df)

1336

In [110]:
%load_ext line_profiler
import line_profiler
#Set compiler directives (cf. http://docs.cython.org/src/reference/compilation.html)
from Cython.Compiler.Options import directive_defaults

directive_defaults['linetrace'] = True
directive_defaults['binding'] = True

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [100]:
%%cython -a -f --compile-args=-DCYTHON_TRACE=1
cimport numpy as np
import cython
import numpy as np

DTYPE = np.float
ctypedef np.float64_t DTYPE_t
import pandas as pd


@cython.boundscheck(False) # turn off bounds-checking for entire function
@cython.wraparound(False)  # turn off negative index wrapping for entire function
def get_quote_fast(dfg, timestamps_list):

    cdef DTYPE_t _last_px
    cdef np.uint64_t[:] npdatetime = dfg.index.values.astype('datetime64[s]').view(np.uint64)

    cdef np.ndarray[DTYPE_t, ndim=2] data = dfg.data

    cdef int ic = dfg.cols['c']
    cdef int count = data.shape[1]
    cdef int i = 0
    cdef int ts_idx = 0
    values = []
    
    if len(timestamps_list) == 0:
        return []
    
    dates = []
    for t in timestamps_list:
        # Make sure that timezones are equal
        if t.tzinfo.zone != dfg.tz.zone:
            # TODO: change to argument error
            raise ValueError("timestamps_list's and quotes dataframe timezones are inequal")
        
        dates.append(np.datetime64(t.replace(tzinfo=None)))
    
    # Create Numpy sorted dates array
    cdef np.uint64_t[:] target_timestamps = np.array(dates, dtype='datetime64[s]').view(np.uint64)
        

    cdef np.ndarray[np.int64_t, ndim=1] ts_indexes = np.searchsorted(npdatetime, target_timestamps)
    
    return ts_indexes       



In [127]:
%lprun -f get_quote_fast get_quote_fast(DataFrameGetter(df), [tz.localize(pd.Timestamp('2012-07-15 16:59:00.000'))])

In [118]:
get_quote_fast(DataFrameGetter(df), [tz.localize(pd.Timestamp('2012-07-16 16:59:00.000')),
                                     tz.localize(pd.Timestamp('2012-07-17 16:59:00.000'))])

array([1335, 1336])

In [119]:
df.index[1335]

Timestamp('2012-07-16 16:59:00-0700', tz='US/Pacific')

In [40]:
%timeit df.index.values.astype('datetime64[s]')

The slowest run took 9.25 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 4.18 µs per loop


In [54]:
dfg  = DataFrameGetter(df)

In [115]:
df.index

DatetimeIndex(['2012-07-15 17:00:00-07:00', '2012-07-15 17:01:00-07:00',
               '2012-07-15 17:02:00-07:00', '2012-07-15 17:03:00-07:00',
               '2012-07-15 17:04:00-07:00', '2012-07-15 17:05:00-07:00',
               '2012-07-15 17:06:00-07:00', '2012-07-15 17:07:00-07:00',
               '2012-07-15 17:08:00-07:00', '2012-07-15 17:09:00-07:00',
               ...
               '2012-07-16 16:43:00-07:00', '2012-07-16 16:44:00-07:00',
               '2012-07-16 16:46:00-07:00', '2012-07-16 16:49:00-07:00',
               '2012-07-16 16:50:00-07:00', '2012-07-16 16:51:00-07:00',
               '2012-07-16 16:54:00-07:00', '2012-07-16 16:56:00-07:00',
               '2012-07-16 16:57:00-07:00', '2012-07-16 16:59:00-07:00'],
              dtype='datetime64[ns, US/Pacific]', name='dt', length=1336, freq=None)

In [75]:
v = []
v.append(np.datetime64(tz.localize(pd.Timestamp('2012-07-15 16:59:00.000')).replace(tzinfo=None)))
v.append(np.datetime64(tz.localize(pd.Timestamp('2012-07-15 16:39:00.000')).replace(tzinfo=None)))

In [79]:
np.sort(np.array(v, dtype='datetime64[s]')).view(np.uint64)

array([1342370340, 1342371540], dtype=uint64)

In [89]:
np.searchsorted(df.index.values, tz.localize(pd.Timestamp('2012-07-15 16:59:00.000')).value)

TypeError: invalid type promotion

In [129]:
t.date()

datetime.date(2012, 7, 15)